<a href="https://colab.research.google.com/github/A-Kshitij-S/Aqi_Prediction/blob/main/Aqi_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd

df = pd.read_excel("aqi_data_one_year_final.xlsx", sheet_name="Sheet1")
df["Xylene"] = df["MP-Xylene"] + df["O-Xylene"]

print(df.head())

          From Date           To Date   PM2.5    PM10     NO    NO2    NOx  \
0  01-01-2024 00:00  02-01-2024 00:00  182.82  260.78  11.08  33.10  25.92   
1  02-01-2024 00:00  03-01-2024 00:00  197.14  298.04  12.58  32.11  26.79   
2  03-01-2024 00:00  04-01-2024 00:00  193.88  269.78  12.41  34.24  27.50   
3  04-01-2024 00:00  05-01-2024 00:00  205.37  285.42  15.46  32.38  29.43   
4  05-01-2024 00:00  06-01-2024 00:00  141.38  217.39  10.62  30.61  24.19   

     NH3    SO2    CO  ...  Eth-Benzene  MP-Xylene  O-Xylene   Temp     RH  \
0  30.31   6.79  1.59  ...         4.92       4.47      4.38  11.08  78.64   
1  30.42  11.08  1.59  ...         4.66       4.13      3.88  10.20  77.01   
2  30.01   8.47  1.69  ...         4.57       4.10      3.87  10.16  89.02   
3  33.33  12.82  1.06  ...         4.61       4.12      3.90   9.41  97.13   
4  48.59  10.43  1.75  ...         3.43       2.76      2.52  11.25  96.53   

     WS      WD     SR  BP  Xylene  
0  0.68  243.46  42.67 Na

In [28]:
from google.colab import files

# Opens a file upload dialog
uploaded = files.upload()

# Access uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')



Saving aqi_data_one_year_final.xlsx to aqi_data_one_year_final.xlsx
Uploaded file: aqi_data_one_year_final.xlsx


In [34]:
def main(filepath, sheet_name=0):
    print(f"Reading Excel file: {filepath}")
    df = pd.read_excel(filepath, sheet_name=sheet_name)
    print(f"Successfully loaded file with shape {df.shape}")

    # Convert numeric columns to proper numeric types
    for col in df.columns:
        if col not in ['From Date', 'To Date']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define AQI calculation (placeholder logic — replace with your actual method)
    def calculate_aqi(row):
        pm25 = row['PM2.5']
        if pm25 <= 30:
            return 50
        elif pm25 <= 60:
            return 100
        else:
            return 200

    # Create AQI column
    df['AQI'] = df.apply(calculate_aqi, axis=1)

    print(f"Loaded and filtered data. Shape: {df.shape}")
    features = ['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Ozone', 'Benzene', 'MP-Xylene']
    print(f"Features used: {features}")
    print(f"Target variable: AQI")

    X = df[features]
    y = df['AQI']

    # The rest of your code...


In [35]:
pip install --upgrade scikit-learn


In [38]:

import os
import argparse
import sys
from zipfile import BadZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Try to import xgboost; fall back gracefully if not installed
try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

# ---------- Configuration ----------
# Define the specific features you want to use for prediction.
FEATURE_COLUMNS = [
    'PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Ozone', 'Benzene', 'MP-Xylene'
]

# Define the column you want to predict.
TARGET_COLUMN = 'AQI'
OUTPUT_DIR = 'output'
RANDOM_STATE = 42
TEST_SIZE = 0.2
# -----------------------------------

def ensure_output_dir():
    """Creates the output directory if it doesn't exist."""
    os.makedirs(OUTPUT_DIR, exist_ok=True)

import pandas as pd
import sys
from zipfile import BadZipFile

import pandas as pd

def load_and_select_features(path, sheet_name=None):
    try:
        print(f"Reading Excel file: {path}")
        df = pd.read_excel(path, sheet_name=sheet_name, engine='openpyxl')
        print(f"Successfully loaded file with shape {df.shape}")
        return df
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        raise



def build_preprocessor():
    """Build a preprocessing pipeline for the numeric features."""
    # This includes imputing missing values (e.g., with the median) and scaling.
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    return numeric_transformer

def train_and_evaluate(X_train, X_test, y_train, y_test):
    """Trains multiple models and returns their performance metrics."""
    models = {
        'LinearRegression': LinearRegression(),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE)
    }
    if XGBOOST_AVAILABLE:
        models['XGBoost'] = XGBRegressor(n_estimators=100, random_state=RANDOM_STATE, verbosity=0)

    results = {}
    print("-" * 30)
    for name, model in models.items():
        # Create a full pipeline including the preprocessor and the model
        pipeline = Pipeline(steps=[
            ('preprocessor', build_preprocessor()),
            ('regressor', model)
        ])

        print(f"Training {name}...")
        pipeline.fit(X_train, y_train)
        preds = pipeline.predict(X_test)

        mae = mean_absolute_error(y_test, preds)
        rmse = mean_squared_error(y_test, preds) ** 0.5

        r2 = r2_score(y_test, preds)

        results[name] = {
            'pipeline': pipeline,
            'predictions': preds,
            'mae': mae,
            'rmse': rmse,
            'r2': r2
        }
        print(f"  -> {name}: MAE={mae:.3f}, RMSE={rmse:.3f}, R2={r2:.3f}")
    print("-" * 30)
    return results

# ---------- Plotting Functions ----------

def plot_true_vs_pred(y_true, y_pred, title, filename):
    plt.figure(figsize=(6, 6))
    plt.scatter(y_true, y_pred, alpha=0.6, edgecolors='k', s=50)
    plt.xlabel('True AQI', fontsize=12)
    plt.ylabel('Predicted AQI', fontsize=12)
    plt.title(title, fontsize=14)
    # Add a 45-degree line for reference (perfect prediction)
    lims = [min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])]
    plt.plot(lims, lims, 'r--', alpha=0.75, zorder=0)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    out_path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(out_path)
    plt.close()
    print(f"Saved plot: {out_path}")

def plot_residuals(y_true, y_pred, title, filename):
    residuals = y_true - y_pred
    plt.figure(figsize=(8, 5))
    plt.scatter(y_pred, residuals, alpha=0.6, edgecolors='k', s=50)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Predicted AQI', fontsize=12)
    plt.ylabel('Residual (True - Predicted)', fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    out_path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(out_path)
    plt.close()
    print(f"Saved plot: {out_path}")

def plot_feature_importance(pipeline, feature_names, title, filename, top_n=15):
    model = pipeline.named_steps['regressor']
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1][:top_n]

        plt.figure(figsize=(10, max(6, len(indices) * 0.4)))
        plt.title(title, fontsize=14)
        plt.barh(range(len(indices)), importances[indices][::-1], align='center')
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices[::-1]])
        plt.xlabel('Relative Importance')
        plt.tight_layout()
        out_path = os.path.join(OUTPUT_DIR, filename)
        plt.savefig(out_path)
        plt.close()
        print(f"Saved plot: {out_path}")
    else:
        print(f"Model {type(model).__name__} does not support feature importances. Skipping plot.")

# ---------- Main Execution Logic ----------

def main(filepath, sheet_name=0):
    print(f"Reading Excel file: {filepath}")
    df = pd.read_excel(filepath, sheet_name=sheet_name)
    print(f"Successfully loaded file with shape {df.shape}")

    # Convert all but the date columns to numeric
    for col in df.columns:
        if col not in ['From Date', 'To Date']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    ensure_output_dir()

    target_columns = [col for col in df.columns if col not in ['From Date', 'To Date']]
    feature_columns = [col for col in target_columns]  # start with all numeric columns

    # Train for each target variable
    for target in target_columns:
        print(f"\n=== Processing target: {target} ===")

        # Remove rows where target is NaN
        df_target = df.dropna(subset=[target])

        X = df_target[[col for col in feature_columns if col != target]]
        y = df_target[target]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
        )

        # Skip if we don't have enough samples
        if len(y_train) < 5 or len(y_test) < 5:
            print(f"⚠ Skipping {target} due to insufficient non-NaN data.")
            continue

        results = train_and_evaluate(X_train, X_test, y_train, y_test)



        for name, res in results.items():
            pipeline = res['pipeline']
            preds = res['predictions']

            out_df = pd.DataFrame({'True': y_test, 'Predicted': preds})
            out_df.to_csv(os.path.join(OUTPUT_DIR, f'predictions_{target}_{name}.csv'), index=False)

            plot_true_vs_pred(y_test, preds,
                              f'True vs. Predicted ({target}, {name})',
                              f'true_vs_pred_{target}_{name}.png')
            plot_residuals(y_test, preds,
                           f'Residuals Plot ({target}, {name})',
                           f'residuals_{target}_{name}.png')

            if name in ['RandomForest', 'GradientBoosting', 'XGBoost']:
                plot_feature_importance(
                    pipeline, X.columns.tolist(),
                    f'Feature Importance ({target}, {name})',
                    f'feature_importance_{target}_{name}.png'
                )

        summary_rows = [
            {'Model': name, 'MAE': res['mae'], 'RMSE': res['rmse'], 'R2': res['r2']}
            for name, res in results.items()
        ]
        summary_df = pd.DataFrame(summary_rows).sort_values('RMSE')
        summary_df.to_csv(os.path.join(OUTPUT_DIR, f'results_summary_{target}.csv'), index=False)

        print("\n--- Model Performance Summary ---")
        print(summary_df.to_string(index=False))


if __name__ == "__main__":
    filepath = "aqi_data_one_year_final.xlsx"  # put the actual file path here
    sheet = 0  # or use a string like "Sheet1"
    main(filepath, sheet_name=sheet)





Reading Excel file: aqi_data_one_year_final.xlsx
Successfully loaded file with shape (739, 22)

=== Processing target: PM2.5 ===
------------------------------
Training LinearRegression...
  -> LinearRegression: MAE=31.871, RMSE=46.059, R2=0.476
Training RandomForest...
  -> RandomForest: MAE=11.607, RMSE=21.523, R2=0.886
Training GradientBoosting...
  -> GradientBoosting: MAE=10.200, RMSE=17.950, R2=0.920
Training XGBoost...
  -> XGBoost: MAE=9.700, RMSE=17.445, R2=0.925
------------------------------
Saved plot: output/true_vs_pred_PM2.5_LinearRegression.png
Saved plot: output/residuals_PM2.5_LinearRegression.png
Saved plot: output/true_vs_pred_PM2.5_RandomForest.png
Saved plot: output/residuals_PM2.5_RandomForest.png
Saved plot: output/feature_importance_PM2.5_RandomForest.png
Saved plot: output/true_vs_pred_PM2.5_GradientBoosting.png
Saved plot: output/residuals_PM2.5_GradientBoosting.png
Saved plot: output/feature_importance_PM2.5_GradientBoosting.png
Saved plot: output/true_vs_pr